In [28]:
import pandas as pd
import numpy as np
from biopsy_grade_extractor import BiopsyGradeExtractor # load exttractor

### load the dataset which contrains biopsy reports

In [2]:
df_pathology = pd.read_csv('../data/sample_biopsy_reports.csv')	

In [33]:
df_pathology_test = pd.read_csv('../data/data.csv')	

In [34]:
df_pathology_test = df_pathology_test.sample(1000)

In [35]:
df_pathology_test

,EMPI,EPIC_PMRN,MRN_Type,MRN,Report_Number,Report_Date_Time,Report_Description,Report_Status,Report_Type,Report_Text,...,num_pos_cores_sum,num_total_core_sum,max_core_involve,small_cell_carc,neuroendocrine_carc,adenocarcinoma,auxiiliary_mci_score,rp_indicator,rp_date,rp_date_minus_report_date_in_days
878,104008446,1.006051e+10,BWH,17205642,S0130184K,9/27/2001 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: S0130184K Report Status: Fin...,...,2,4,"[[5], [5]]",0,0,1,0.10,1,2002-03-27 00:00:00,181.0
885,104130958,1.006384e+10,MGH,4431904,S19-60024,8/28/2019 9:09:00 AM,Surgical Pathology,Final,PAT,Accession Number: S19-60024 Report Status: Fin...,...,3,5,"[[10], [40]]",0,0,1,1.80,1,2019-10-21 00:00:00,53.0
14734,106766874,1.004915e+10,BWH,18949131,BS07D56675,12/21/2007 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS07D56675 Report Status: Fi...,...,1,5,[[5]],0,0,1,0.05,0,NaN,NaN
14290,106629297,1.004352e+10,BWH,26477026,BS11X56755,12/9/2011 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS11X56755 Report Status: Fi...,...,1,6,[[5]],0,0,1,0.05,0,NaN,NaN
12237,103535316,1.002323e+10,MGH,1622402,MS10F80363,12/8/2010 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: MS10F80363 Report Status: Fi...,...,12,12,"[[70], [70], [60], [60], [90], [70], [95], [90...",0,0,1,20.15,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14058,113699536,1.012749e+10,BWH,36767606,BS18R09386,2/11/2013 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS18R09386 Report Status: Fi...,...,2,12,"[[5], [10]]",0,0,1,0.15,0,NaN,NaN
17900,109925513,1.006928e+10,BWH,15003940,S9831032W,11/23/1998 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: S9831032W Report Status: Fin...,...,1,4,[[5]],0,0,1,0.05,0,NaN,NaN
11788,108880595,1.009142e+10,BWH,28967156,BS14R01741,1/13/2014 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS14R01741 Report Status: Fi...,...,3,4,"[[10], [10, 5]]",0,0,1,0.55,0,NaN,NaN
16077,104594693,1.007855e+10,BWH,22490353,BS07D55019,12/12/2007 12:00:00 AM,Surgical Pathology,Final,PAT,Accession Number: BS07D55019 Report Status: Fi...,...,2,3,"[[30, 30]]",0,0,1,0.60,0,NaN,NaN


### Instantiate BiopsyGradeExtractor and fit the data

In [36]:
extractor = BiopsyGradeExtractor()
df_result = extractor.fit(df_pathology_test, display_summary = True, return_summary_df = True)

extracing info...: 100%|██████████| 1000/1000 [00:00<00:00, 4459.35it/s]



Extraction complete.


Displaying summary...
Counts per each primary grade: 
3.0    1757
4.0     611
5.0      70
3.5      13
2.0       8
dtype: int64


Counts per each secondary grade: 
3.0    1465
4.0     766
5.0     204
3.5      13
2.0      11
dtype: int64


Number of reports per each overall grade : 
1.0    443
2.0    263
3.0    126
5.0     86
4.0     82
dtype: int64
total =  1000
Number of unique biopsy reports :  1000
Number of unique patients :  1000
Number of unique patients with overall_grade_merged :  1000



/Users/guangyawan/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3199: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
/Users/guangyawan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/internals/blocks.py:937: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.asarray(value)


In [46]:
df_result[['Report_Text','overall_grade_merged']].sample(100).to_excel('sample_output.xlsx',index = False)

In [21]:
df_result.overall_grade_merged.iloc[0]

In [10]:
df_pathology.iloc[1]['Report_Text']

NameError: name 'df_pathology' is not defined

### Validate the performance of the algorithm

In [5]:
# if report_number_oi is not specified, then it randomly picks report to evaluate
for i in range(2):
    extractor.validate_results()



Extracted info : 
Report_Number                                                  SPNPSGOSBB
Report_Text             LEFT (A3): Benign prostate tissue. LEFT MID (B...
primary_grade                                                         [3]
secondary_grade                                                       [3]
overall_grade_merged                                                  1.0
Name: 17, dtype: object


Pathology report : 
LEFT (A3): Benign prostate tissue. LEFT MID (B3): Benign prostate tissue. LEFT APEX (C3): Benign prostate tissue. RIGHT BASE (D3): Benign prostate tissue. RIGHT MID (E3): PROSTATIC ADENOCARCINOMA, Gleason grade 3+3, involving 1 of 1 core, approximately 15% of tissue. RIGHT APEX (F3): Benign prostate tissue.




Extracted info : 
Report_Number                                                  SPNPSGOSBB
Report_Text             LEFT (A3): Benign prostate tissue. LEFT MID (B...
primary_grade                                                         [3]
secondary_g

In [ ]:
# Primary Grade/Secondary Grade/overall grade merged

In [6]:
df_result

,Unnamed: 0,EMPI,MRN_Type,Report_Number,Report_Text,primary_grade,secondary_grade,overall_grade_merged
0,0,5162687835,MGH,YDOXSRZFJF,Firm right prostate lobe. ====================...,[3],[3],1.0
1,1,6545694958,MGH,PWDMXHIVGX,"A. PROSTATE NEEDLE BIOPSIES, RIGHT BASE: PROST...","[4, 4, 4, 3, 3, 3]","[4, 4, 3, 4, 4, 3]",4.0
2,2,7804781214,BWH,KFDPJIOOSD,A) RIGHT LOBE: Benign prostatic tissue. Immuno...,[4],[3],3.0
3,3,480085707,BWH,KKOAKIFZSY,PROSTATE; NEEDLE BIOPSIES : A) LEFT SIDE; PROS...,"[3, 3]","[3, 3]",1.0
4,4,6853020993,BWH,FMPNJLHJYW,"RIGHT PROSTATE, NEEDLE BIOPSY: PROSTATIC ADENO...","[5, 5]","[5, 5]",5.0
5,5,7042372336,BWH,HQZQSVIBQE,A) LEFT APEX: Benign prostatic tissue. B) LEFT...,"[4, 3, 3, 3]","[3, 4, 3, 4]",3.0
6,6,1134543123,BWH,TCHTAECHHT,CORE NEEDLE BIOPSIES (XXX): A) RIGHT APEX: Ben...,"[3, 3]","[3, 4]",2.0
7,7,4022503058,MGH,MSWIDQDSFI,"A. PROSTATE, RIGHT BASE: PROSTATIC ADENOCARCIN...",[3],[3],1.0
8,8,2586526426,BWH,SMOEUKNPBZ,A. RIGHT APEX (BIOPSY/IES): Atypical small aci...,"[4, 3, 4]","[3, 4, 3]",3.0
9,9,4954105822,MGH,DGPIVXRPDB,"A. PROSTATE, RIGHT 4 MID (S13-09075-H; 02/27/2...","[3, 3, 3]","[3, 3, 3]",1.0


In [3]:
!wget -O pat3 https://www.dropbox.com/s/kqjwb9nh130bz37/Pat.csv?dl=0

zsh:1: no matches found: https://www.dropbox.com/s/kqjwb9nh130bz37/Pat.csv?dl=0
